# Google Natural Questions

## Introduction

Le but de ce challenge est de créer un modèle d'apprentissage statistique pour répondre aux questions posées en se servant d'un corps de texte (i.e. Wikipedia). Pour accéder au lien du *challenge*, veuillez clicquez [ici](https://ai.google.com/research/NaturalQuestions).

## Présentation des données

Dans cette partie nous présentons les données fournies. Le format des données d'apprentissage est `json`. En revanche Google a créé une version tabulée des données qui se trouve dans ce [lien](https://ai.google.com/research/NaturalQuestions/databrowser).

Nous utiliserons les données simplifiées par Google:

- **simplified-nq-train.jsonl**: 17.45 Go
- **simplified-nq-test.jsonl**: 18.8 Mo

La base d'apprentissage étant assez volumineuse, nous utiliserons le Google Cloud Platform pour nos calculs.

Ci-dessous, les nombres d'observations dans les bases fournies par Google:

- **Base d'apprentissage:** 307 373 observations
- **Base de validation (ou de développement):** 7 830 observations
- **Base de test:** 7 842 observations

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

local = "local[*]"
appName = "QA"
configLocale = SparkConf().setAppName(appName).setMaster(local).\
set("spark.executor.memory", "6G").\
set("spark.driver.memory", "6G").\
set("spark.sql.catalogImplementation", "in-memory")

spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
spark

Afin d'inspecter le format du fichier `json`nous avons créé une version courte, que nous chargeons ici.

In [20]:
for_content = spark.read.json("/Volumes/750GB-HDD/root/Question-Answering/pyData/tensorflow2-question-answering/simplified-nq-train-for-content.json")

In [18]:
for_content.printSchema()

root
 |-- annotations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotation_id: long (nullable = true)
 |    |    |-- long_answer: struct (nullable = true)
 |    |    |    |-- candidate_index: long (nullable = true)
 |    |    |    |-- end_token: long (nullable = true)
 |    |    |    |-- start_token: long (nullable = true)
 |    |    |-- short_answers: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_token: long (nullable = true)
 |    |    |    |    |-- start_token: long (nullable = true)
 |    |    |-- yes_no_answer: string (nullable = true)
 |-- document_text: string (nullable = true)
 |-- document_url: string (nullable = true)
 |-- example_id: long (nullable = true)
 |-- long_answer_candidates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- end_token: long (nullable = true)
 |    |    |-- start_token: long (nullable = true)
 |    |    |

## Les variables

### Annotations

Cette variable contient réponses **longues** et **courtes**, s'il existe des réponses. Ci-dessous les règles de cette variable:

- Chaque question a une réponse longue au maximum. En revanche, il peut y avoir plusieurs réponses courtes.
- Les réponses courtes sont nécessairement contenues dans la réponse longue. Si la réponse est de type **Oui/Non**, alors `yes_no_answer` prend les valeurs `Yes` ou `No`. Par défaut, la valeur de cette variable est `None`.
- Seulement 1% des réponses sont de type **Oui/Non**.

### Document text
Cette variable contient le corps de la page Wikipedia en format *html*.

### Document URL
Cette variable contient le lien URL vers la page Wikipedia.

### Example ID
C'est l'identifiant de l'exemple.

### Long answer candidates
Cette variable contient les réponses candidates. 

- Parfois, une longue réponse est imbriquée dans une autre.
- Pour différencier ces deux types de réponses, on utilise la notion de **niveau**. Une réponse est donc contenue dans une autre si son indicateur *top level* est `False`.
- 95% des réponses longues sont du *top level* `True`. Nous pourrions donc, dans un premier temps, nous focaliser sur ces réponses uniquement.

### Question text
C'est la question posée.

In [29]:
for_content.annotations

Column<b'annotations'>

In [22]:
for_content_2 = spark.read.json("/Volumes/750GB-HDD/root/Question-Answering/pyData/tensorflow2-question-answering/v1.0-simplified_nq-dev-all-for-content.json")

In [24]:
for_content_2.printSchema()

root
 |-- annotations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotation_id: decimal(20,0) (nullable = true)
 |    |    |-- long_answer: struct (nullable = true)
 |    |    |    |-- candidate_index: long (nullable = true)
 |    |    |    |-- end_byte: long (nullable = true)
 |    |    |    |-- end_token: long (nullable = true)
 |    |    |    |-- start_byte: long (nullable = true)
 |    |    |    |-- start_token: long (nullable = true)
 |    |    |-- short_answers: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_byte: long (nullable = true)
 |    |    |    |    |-- end_token: long (nullable = true)
 |    |    |    |    |-- start_byte: long (nullable = true)
 |    |    |    |    |-- start_token: long (nullable = true)
 |    |    |-- yes_no_answer: string (nullable = true)
 |-- document_html: string (nullable = true)
 |-- document_title: string (nullable = true)
 |-- document_to